In [2]:
import pandas as pd
import numpy as np

## Универсальные функции: сохранение индекса

In [4]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [5]:
df = pd. DataFrame(rng.randint(0, 10, (3, 4)), 
                   columns=['a', 'b', 'c', 'd'])
df

,a,b,c,d
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [6]:
np.log10(ser) # Можно применять функции NumPy к объектам Pandas и индекс сохранится

0    0.778151
1    0.477121
2    0.845098
3    0.602060
dtype: float64

In [7]:
np.sin(df * np.pi / 4) # Даже более сложные вычисления

,a,b,c,d
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## Универсальные функции: выравнивание индексов

In [6]:

area = pd.Series({'Alaska': 1723337, 'Texas': 695662, 'California': 423967}, name='area') 
population = pd.Series({'California': 38332521, 'Texas': 26448193, 'New York': 19651127}, name='population')
area / population

Alaska             NaN
California    0.011060
New York           NaN
Texas         0.026303
dtype: float64

In [7]:
# NaN получается в случае отсутствия данных в одном из массивов по какому-то индексу
print(area.index & population.index)
print(area.index | population.index)

Index(['Texas', 'California'], dtype='object')
Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')


In [8]:
area.div(population, fill_value=population.mean()) # Заполнение пустых значений

Alaska        0.061233
California    0.011060
New York      1.432180
Texas         0.026303
dtype: float64

In [9]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [10]:
B = pd.DataFrame(rng.randint(0, 20, (3, 3)), columns=list('BCA'))
B

,B,C,A
0,0,11,11
1,16,9,15
2,14,14,18


In [11]:
# При вы полнении операций DataFrame происходит выравнивание индексов и столбцов
A + B   # причем индексы выравниваются правильно не смотря на их изначальное положение

,A,B,C
0,12.0,11.0,NaN
1,20.0,17.0,NaN
2,NaN,NaN,NaN


In [12]:
fill = A.stack().mean()   
A.add(B, fill_value=fill)

,A,B,C
0,12.0,11.0,15.5
1,20.0,17.0,13.5
2,22.5,18.5,18.5


In [13]:
A.stack() # Функция stack() выравнивает по одному измерению с помощью MultiIndex

0  A     1
   B    11
1  A     5
   B     1
dtype: int32

## Универсальные операции: выполнение операций между DataFrame и Series

In [14]:
A_np = np.array(A)
A_np

array([[ 1, 11],
       [ 5,  1]])

In [15]:
A_np - A_np[0]  # часто встречающаяся операция вычитания из массива одной из его строк

array([[  0,   0],
       [  4, -10]])

In [16]:
A - A.iloc[0]  # автоматически из строк

,A,B
0,0,0
1,4,-10


In [17]:
A.subtract(A['B'], axis=0) # Если надо по столбцам

,A,B
0,-10,0
1,4,0


In [18]:
B.iloc[0, ::2]  # A и B, т.к. порядок в DataFrame такой

B     0
A    11
Name: 0, dtype: int32

In [19]:
B

,B,C,A
0,0,11,11
1,16,9,15
2,14,14,18


In [20]:
B - B.iloc[0, ::2]

,A,B,C
0,0.0,0.0,NaN
1,4.0,16.0,NaN
2,7.0,14.0,NaN


## Отсутствующие данные 

In [21]:
val = np.array([1, None, 3, 4])  #Если поппытаться сделать массив Numpy из списка с None значениями, 
val              #то для массива будет выбран тип object, что очень замедлит его работу.

array([1, None, 3, 4], dtype=object)

In [22]:
val.sum()       # выдает ошибку из-за None

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [23]:
val2 = np.array([1, np.nan, 3, 4])   # Но при NaN значениях все работает хорошо
print(val2, val2.dtype)              # тип массива уже число с плавающей точкой, т.к. у NaN такой тип
print(val2.min())                    # NaN как Null все заражает
print(np.nanmax(val2))               # для этого есть функции, которые его игнорируют 

[ 1. nan  3.  4.] float64
nan
4.0


In [24]:
val_pd = pd.Series([1, np.nan, 3, None])  # Pandas автоматически преобразовывает в NaN все пустые значения
val_pd

0    1.0
1    NaN
2    3.0
3    NaN
dtype: float64

In [25]:
val_int = pd.Series([1, 2, 3]) 
val_bool = pd.Series([True, False])
print(np.dtype(val_int))
print(np.dtype(val_bool))

int64
bool


In [26]:
val_int[0] = None       # вставка NaN автоматически повышает тип
val_bool[0] = np.nan
print(np.dtype(val_int))
print(np.dtype(val_bool))  # хз вообще пишут что должно быть object из bool))

float64
float64


### Обработка пустых значений

In [27]:
print(val_pd)
print()
print(val_pd.isnull())
print()
print(val_pd.notnull())


0    1.0
1    NaN
2    3.0
3    NaN
dtype: float64

0    False
1     True
2    False
3     True
dtype: bool

0     True
1    False
2     True
3    False
dtype: bool


In [28]:
#Удаление пустых значений из Series
print(val_pd[val_pd.notnull()])
print()
print(val_pd.dropna())

0    1.0
2    3.0
dtype: float64

0    1.0
2    3.0
dtype: float64


In [29]:
df = pd.DataFrame(np.random.randint(1, 7, (3, 3)))
df.iloc[0, 1] = None
df.iloc[2, 0] = np.nan
df

,0,1,2
0,6.0,NaN,3
1,5.0,3.0,6
2,NaN,4.0,5


In [30]:
# Но в DataFrame надо сбрасывать целые строки или столбцы
df.dropna()  # по автомату строки

,0,1,2
1,5.0,3.0,6


In [31]:
# но можно и столбцы
df.dropna(axis='columns')

,2
0,3
1,6
2,5


In [32]:
df[3] = np.nan
print(df)
print()
# также условия сброса можно контроллировать с помощью thresh и how
print(df.dropna(axis='columns', how='all'))  # по умолчанию how='any', т.е. удаляется любой столбец с пустым значением


     0    1  2   3
0  6.0  NaN  3 NaN
1  5.0  3.0  6 NaN
2  NaN  4.0  5 NaN

     0    1  2
0  6.0  NaN  3
1  5.0  3.0  6
2  NaN  4.0  5


In [33]:
df.dropna(thresh=3)  # thresh - задает минимальное количество НЕпустых значений, 
                     # при котором столбец/строка отбрасываться не будет

,0,1,2,3
1,5.0,3.0,6,NaN


#### Заполнение пустых значений

In [34]:
data = pd.Series([1, np.nan, 2, np.nan, 3])
data

0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
dtype: float64

In [35]:
# Заполнение пустых полей фиксированным значением
data.fillna(0)


0    1.0
1    0.0
2    2.0
3    0.0
4    3.0
dtype: float64

In [36]:
# Заполнение по напралению вперед(копируя предыдущее значение)
data.fillna(method='ffill')

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
dtype: float64

In [37]:
# по направлению назад
data.fillna(method='bfill')

0    1.0
1    2.0
2    2.0
3    3.0
4    3.0
dtype: float64

In [38]:
df # в DataFrame ситуации аналогичны

,0,1,2,3
0,6.0,NaN,3,NaN
1,5.0,3.0,6,NaN
2,NaN,4.0,5,NaN


In [39]:
# только можно задавать направление
df.fillna(method='ffill', axis=1)
# Если предыдущего значения нет, то поле остается не заполненным

,0,1,2,3
0,6.0,6.0,3.0,3.0
1,5.0,3.0,6.0,6.0
2,NaN,4.0,5.0,5.0


# Иерархическая индексация

In [40]:
# Плохой способ создания иерархических индексов
index_tuple = [('California', 2000), ('California', 2001),
               ('New York', 2000), ('New York', 2001),
               ('Texas', 2000),('Texas', 2001)]
population = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]

pop = pd.Series(population, index=index_tuple)
print(pop)
print(pop[[i for i in pop.index if i[1] == 2001]])  # Не удобно делать выборку, да и работает она медленно

(California, 2000)    33871648
(California, 2001)    37253956
(New York, 2000)      18976457
(New York, 2001)      19378102
(Texas, 2000)         20851820
(Texas, 2001)         25145561
dtype: int64
(California, 2001)    37253956
(New York, 2001)      19378102
(Texas, 2001)         25145561
dtype: int64


In [41]:
# Создание MultiIndex
index = pd.MultiIndex.from_tuples(index_tuple)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2001]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [42]:
pop = pop.reindex(index)
print(pop)
print()
print(pop[:, 2001])  # пользуемся стандартной индецксацией Pandas - быстрее и удобнее

California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
Texas       2000    20851820
            2001    25145561
dtype: int64

California    37253956
New York      19378102
Texas         25145561
dtype: int64


In [43]:
# Можно легко перевести этот объект Series в DataFrame
pop_df = pop.unstack()
pop_df

,2000,2001
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [44]:
#  И обратно
pop_df.stack()

California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
Texas       2000    20851820
            2001    25145561
dtype: int64

In [45]:
# MultiIndex позволяет легко обращаться с тремя и более измерениями
pop_df = pd.DataFrame({'total': pop, 
                       'under18': [9267089, 9284094,
                                   4687374, 4318033, 
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2001  37253956  9284094
New York   2000  18976457  4687374
           2001  19378102  4318033
Texas      2000  20851820  5906301
           2001  25145561  6879014

In [46]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2001
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### Создание Мультииндексов

In [47]:
# Два простых метода создания
# Список списков
df = pd.DataFrame(np.random.rand(4, 2), 
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.056890  0.698203
  2  0.593486  0.038650
b 1  0.713929  0.079053
  2  0.888516  0.813845

In [48]:
# Просто передаем Словарь с ключами-кортежами
data = {k: i for k in index_tuple for i in population}
print(data)
pd.Series(data)

{('California', 2000): 25145561, ('California', 2001): 25145561, ('New York', 2000): 25145561, ('New York', 2001): 25145561, ('Texas', 2000): 25145561, ('Texas', 2001): 25145561}


California  2000    25145561
            2001    25145561
New York    2000    25145561
            2001    25145561
Texas       2000    25145561
            2001    25145561
dtype: int64

#### Явные конструкторы MultiIndex

In [49]:
# Из списка массивов вручную
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])


MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [50]:
# Из списка кортежей
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [51]:
# Из декартова произведения 
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [52]:
# С помощью стандартного предситавления
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], 
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

### Название уровней мультииндексов

In [53]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
Texas       2000    20851820
            2001    25145561
dtype: int64

### Мультииндекс для столбцов

In [54]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  37.3  39.0  38.3  48.0  37.0
     2      31.0  37.4  39.0  38.4  21.0  36.6
2014 1      40.0  37.7  36.0  37.3  31.0  38.4
     2      38.0  36.5  46.0  36.7  34.0  34.8

In [55]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      39.0  38.3
     2      39.0  38.4
2014 1      36.0  37.3
     2      46.0  36.7

### Мультииндексация объектов Series

In [56]:
pop

state       year
California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
Texas       2000    20851820
            2001    25145561
dtype: int64

In [57]:
pop['California', 2001]

37253956

In [58]:
pop['California']

year
2000    33871648
2001    37253956
dtype: int64

In [59]:
# Выполнение частичных срезов, если массив отсортирован
pop.loc['California':'New York']

state       year
California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
dtype: int64

In [60]:
# Индексация по нижним индексам, тоже в отсортированном
pop[:, 2001]

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [61]:
# Булева маска
pop[pop > 21000000]

state       year
California  2000    33871648
            2001    37253956
Texas       2001    25145561
dtype: int64

In [62]:
# "прихотливая" индексация
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2001    37253956
Texas       2000    20851820
            2001    25145561
dtype: int64

### Мультииндексация объектов DataFrame

In [63]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  37.3  39.0  38.3  48.0  37.0
     2      31.0  37.4  39.0  38.4  21.0  36.6
2014 1      40.0  37.7  36.0  37.3  31.0  38.4
     2      38.0  36.5  46.0  36.7  34.0  34.8

In [64]:
health_data['Guido', 'HR']

year  visit
2013  1        39.0
      2        39.0
2014  1        36.0
      2        46.0
Name: (Guido, HR), dtype: float64

In [65]:
# необходимо указывать кортеж из индексов
health_data.loc[:, (['Guido', 'Bob'], 'Temp')]

subject      Bob Guido
type        Temp  Temp
year visit            
2013 1      37.3  38.3
     2      37.4  38.4
2014 1      37.7  37.3
     2      36.5  36.7

In [66]:
health_data[health_data[]]

SyntaxError: invalid syntax (<ipython-input-66-21bbf3a4e920>, line 1)

In [67]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      34.0  37.3
     2      31.0  37.4

In [68]:
# срезы формируют явным образом с помощью IndexSlice
idx = pd.IndexSlice
health_data.loc[: ,idx[:, 'Temp']]

subject      Bob Guido   Sue
type        Temp  Temp  Temp
year visit                  
2013 1      37.3  38.3  37.0
     2      37.4  38.4  36.6
2014 1      37.7  37.3  38.4
     2      36.5  36.7  34.8

In [69]:
health_data.loc[idx[:, 2] ,:]

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 2      31.0  37.4  39.0  38.4  21.0  36.6
2014 2      38.0  36.5  46.0  36.7  34.0  34.8

In [70]:
health_data.loc[idx[:, 2], idx[:, 'Temp']]

,subject,Bob,Guido,Sue
,type,Temp,Temp,Temp
year,visit,,,
2013,2,37.4,38.4,36.6
2014,2,36.5,36.7,34.8


In [71]:
health_data[health_data.loc[: ,idx[:, 'Temp']]> 38]
# Странную фигню понаделал)

subject    Bob      Guido       Sue      
type        HR Temp    HR  Temp  HR  Temp
year visit                               
2013 1     NaN  NaN   NaN  38.3 NaN   NaN
     2     NaN  NaN   NaN  38.4 NaN   NaN
2014 1     NaN  NaN   NaN   NaN NaN  38.4
     2     NaN  NaN   NaN   NaN NaN   NaN

### Перегруппировка МультиИндексов

In [72]:
# Создадим новый Индекс - он не отсортирован лексиграфически
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']

data

char  int
a     1      0.851757
      2      0.789261
c     1      0.807102
      2      0.521385
b     1      0.282427
      2      0.940647
dtype: float64

In [73]:
# Если пытаться сделать частичный срез
try:
    data['a': 'b'] # Получаем ошибку, из-за неотсортированного индекса
except KeyError as e:
    print(e)

'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [74]:
# Отсортировать индекс можно следующим образом
data.sort_index()['a':'b']


char  int
a     1      0.851757
      2      0.789261
b     1      0.282427
      2      0.940647
dtype: float64

## Stack, unstack над индексами

In [75]:
pop

state       year
California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
Texas       2000    20851820
            2001    25145561
dtype: int64

In [76]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2001,37253956,19378102,25145561


In [77]:
pop.unstack(level=1)

year,2000,2001
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [78]:
# Методу unstack противоположен метод Stack
pop.unstack().stack()

state       year
California  2000    33871648
            2001    37253956
New York    2000    18976457
            2001    19378102
Texas       2000    20851820
            2001    25145561
dtype: int64

## Удобное создание и перестройка MultiIndex

In [79]:
# Индекс всегда можно передать опять столбец\
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2001,37253956
2,New York,2000,18976457
3,New York,2001,19378102
4,Texas,2000,20851820
5,Texas,2001,25145561


In [80]:
# для того чтобы удобно создать индекс в реальных данных используют функцию set_index
pop_flat.set_index(['state', 'population'])


year
state      population      
California 33871648    2000
           37253956    2001
New York   18976457    2000
           19378102    2001
Texas      20851820    2000
           25145561    2001

## Агрегирование по МультиИндексам

In [81]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  37.3  39.0  38.3  48.0  37.0
     2      31.0  37.4  39.0  38.4  21.0  36.6
2014 1      40.0  37.7  36.0  37.3  31.0  38.4
     2      38.0  36.5  46.0  36.7  34.0  34.8

In [93]:
health_data.max()

subject  type
Bob      HR      40.0
         Temp    37.7
Guido    HR      46.0
         Temp    38.4
Sue      HR      48.0
         Temp    38.4
dtype: float64

In [94]:
# Для уточнения конкретных строк при использовании методов агрегирования
# по которым будет проводиться используется параметр level
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     32.5  37.35  39.0  38.35  34.5  36.8
2014     39.0  37.10  41.0  37.00  32.5  36.6

In [96]:
# для выбора индекса агрегирования по столбцам используется axis
data_mean.min(axis=1, level='type')

type,HR,Temp
year,,
2013,32.5,36.8
2014,32.5,36.6


In [ ]:
# Для трехмерных и четырехмерных данных в Pandas есть pd.Panel и pd.Panel4D соответственно,
# но вполне может хватить МультиИндексов